<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

### 1112. Highest Grade For Each Student

```mysql
-- MYSQL
SELECT student_id, MIN(course_id) AS course_id, grade
FROM (
    SELECT e.student_id, e.course_id, e.grade
    FROM enrollments e
    JOIN (
        SELECT student_id, MAX(grade) AS grade
        FROM enrollments
        GROUP BY student_id) x
    ON e.student_id=x.student_id AND e.grade=x.grade) y
GROUP BY student_id, grade
ORDER BY student_id

-- SQL server
SELECT student_id, course_id, grade
FROM
(SELECT student_id, course_id, grade,
    RANK() OVER (partition by student_id ORDER BY grade DESC, course_id) rnk
    FROM enrollments
) X
WHERE rnk=1
ORDER BY student_id
```

### 1113. Reported Posts

```mysql
SELECT extra AS report_reason, COUNT( DISTINCT post_id) AS report_count
FROM actions
WHERE action_date='2019-07-04' AND action='report'
GROUP BY extra
```

### 1126. Active Businesses

```mysql
-- 1st submission
/*SELECT business_id
FROM (
    SELECT business_id, event_type, occurences
    FROM events e
    JOIN (
        SELECT event_type AS e_type, AVG(occurences) AS avg_val
        FROM events
        GROUP BY event_type) x
    ON e.event_type=x.e_type AND e.occurences>x.avg_val) y
GROUP BY business_id
HAVING COUNT(*)>1*/

-- 2nd submission
SELECT e1.business_id
FROM
Events e1 LEFT JOIN
(SELECT event_type ,avg(occurences) AS event_avg
FROM Events GROUP BY event_type ) av
ON e1.event_type = av.event_type
GROUP BY business_id
HAVING SUM( CASE WHEN e1.occurences > av.event_avg THEN 1 ELSE 0 END )>1
```

### 1127. User Purchase Platform

```mysql
-- MYSQL
-- use GROUP_CONCAT
SELECT a.spend_date, b.platform, 
    IFNULL(c.total_amount, 0) total_amount,
    IFNULL(c.total_users, 0) total_users
FROM 
    (SELECT DISTINCT spend_date FROM Spending) a
JOIN
(SELECT "desktop" AS platform UNION SELECT "mobile" UNION SELECT "both") b
LEFT JOIN
(
    SELECT spend_date, platform, SUM(amount) total_amount, COUNT(amount) total_users
    FROM (
        SELECT
            user_id,
            spend_date,
            (CASE WHEN LENGTH(GROUP_CONCAT(platform)) = 14 THEN "both" 
                ELSE GROUP_CONCAT(platform) END) platform,
            SUM(amount) amount
        FROM Spending
        GROUP BY user_id, spend_date
    ) p
    GROUP BY spend_date, platform
) c
ON a.spend_date = c.spend_date AND b.platform=c.platform

-- union solution, verbose and slow
/*SELECT s1.spend_date, IFNULL(platform, 'desktop') AS platform,
    IFNULL(total_amount, 0) AS total_amount, IFNULL(total_users, 0) AS total_users
FROM (
    SELECT DISTINCT(spend_date) AS spend_date
    FROM spending) s1
LEFT JOIN (
    SELECT spend_date, 'desktop' AS platform,
        SUM(total_users) AS total_users, SUM(total_amount) AS total_amount
    FROM (
        SELECT spend_date, user_id,
            COUNT(CASE WHEN platform='desktop' THEN user_id END) AS total_users,
            SUM(CASE WHEN platform='desktop' THEN amount END) AS total_amount
        FROM spending
        GROUP BY spend_date, user_id
        HAVING COUNT(DISTINCT platform)=1 AND total_users!=0) x
    GROUP BY spend_date) y
ON s1.spend_date=y.spend_date
UNION
SELECT s1.spend_date, IFNULL(platform, 'mobile') AS platform,
    IFNULL(total_amount, 0) AS total_amount, IFNULL(total_users, 0) AS total_users
FROM (
    SELECT DISTINCT(spend_date) AS spend_date
    FROM spending) s1
LEFT JOIN (
    SELECT spend_date, 'mobile' AS platform,
        SUM(total_users) AS total_users, SUM(total_amount) AS total_amount
    FROM (
        SELECT spend_date, user_id,
            COUNT(CASE WHEN platform='mobile' THEN user_id END) AS total_users,
            SUM(CASE WHEN platform='mobile' THEN amount END) AS total_amount
        FROM spending
        GROUP BY spend_date, user_id
        HAVING COUNT(DISTINCT platform)=1 AND total_users!=0) x
    GROUP BY spend_date) y
ON s1.spend_date=y.spend_date
UNION
SELECT s1.spend_date, IFNULL(platform, 'both') AS platform,
    IFNULL(total_amount, 0) AS total_amount, IFNULL(total_users, 0) AS total_users
FROM (
    SELECT DISTINCT(spend_date) AS spend_date
    FROM spending) s1
LEFT JOIN (
    SELECT s.spend_date, 'both' AS platform, COUNT(DISTINCT s.user_id) AS total_users, SUM(s.amount) AS total_amount
    FROM spending s
    JOIN (
        SELECT spend_date, user_id
        FROM spending
        GROUP BY spend_date, user_id
        HAVING COUNT(DISTINCT platform)>1) x
    ON s.spend_date=x.spend_date AND s.user_id=x.user_id
    GROUP BY spend_date) y
ON s1.spend_date=y.spend_date
ORDER BY spend_date, platform*/
-- SQL server
SELECT date_platform.spend_date, date_platform.platform,
    SUM(CASE WHEN amount IS NULL THEN 0 ELSE amount END) as total_amount,
    COUNT(DISTINCT user_id) AS total_users
FROM(
    SELECT * FROM
    (SELECT distinct(spend_date) FROM spending) a
    CROSS JOIN
    (SELECT distinct(platform) FROM spending union select 'both') b ) date_platform
LEFT JOIN(
    SELECT user_id, spend_date, platform, amount,
    (CASE WHEN cnt_platform=1 THEN platform ELSE 'both' END) AS platform1
    FROM(
        SELECT user_id, spend_date, platform, amount, count(*) over(partition by spend_date, user_id) as cnt_platform
        FROM spending) temp
    ) temp1
ON date_platform.spend_date = temp1.spend_date
    AND date_platform.platform = temp1.platform1
GROUP BY date_platform.spend_date, date_platform.platform
ORDER BY date_platform.spend_date;
```

### 1132. Reported Posts II

```mysql
SELECT Round(AVG(A.rate) *100,2) as average_daily_percent
FROM (
    SELECT action_date, COUNT(DISTINCT r.post_id)/ COUNT(DISTINCT a.post_id) AS rate
    FROM Actions a
    LEFT JOIN Removals r
    ON a.post_id=r.post_id
    WHERE ACTION='report' AND extra='spam'
    GROUP BY action_date) A

/*SELECT ROUND(AVG(daily_percent), 2) AS average_daily_percent 
FROM (
    SELECT action_date, SUM(spam)/COUNT(spam)*100 AS daily_percent
    FROM (
        SELECT action_date, CASE WHEN r.post_id IS NULL THEN 0 ELSE 1 END AS spam, a.extra
        FROM (
            SELECT  * FROM actions
            WHERE action='report' AND extra='spam') a
        LEFT JOIN removals r
        ON a.post_id=r.post_id) b
    GROUP BY action_date) c */
```

### 1141. User Activity for the Past 30 Days I

```mysql
SELECT activity_date AS day, COUNT(DISTINCT user_id) AS active_users
FROM Activity
WHERE activity_date BETWEEN '2019-06-28' AND '2019-07-27'
GROUP BY activity_date
HAVING active_users>0
```

### 1142. User Activity for the Past 30 Days II

```mysql
SELECT ROUND(IFNULL(AVG(n_sessions), 0.0), 2) AS average_sessions_per_user
FROM (
    SELECT activity_date, COUNT(DISTINCT session_id) AS n_sessions
    FROM Activity
    WHERE activity_date BETWEEN '2019-06-28' AND '2019-07-27'
    GROUP BY user_id) x
```

### 1148. Article Views I

```mysql
SELECT DISTINCT author_id AS id
FROM views
WHERE author_id=viewer_id
ORDER BY author_id
```

### 1149. Article Views II

```mysql
-- one query, more concise
SELECT DISTINCT(viewer_id) AS id
FROM views
GROUP BY viewer_id, view_date
HAVING COUNT(DISTINCT article_id)>1
ORDER BY viewer_id

/* SELECT DISTINCT viewer_id AS id
FROM
(
    SELECT viewer_id, view_date, COUNT(DISTINCT article_id) AS articles
    FROM views
    GROUP BY viewer_id, view_date
    HAVING articles>1) a
ORDER BY viewer_id */
```

### 1158. Market Analysis I

```mysql
/* SELECT u.user_id AS buyer_id, u.join_date, IFNULL(x.orders, 0) AS orders_in_2019
FROM users u
LEFT JOIN (
    SELECT buyer_id, COUNT(order_id) AS orders
    FROM orders
    WHERE year(order_date)=2019
    GROUP BY buyer_id) x
ON u.user_id=x.buyer_id*/
-- faster solution
SELECT u.user_id as buyer_id , u.join_date,
    SUM(CASE WHEN YEAR(o.order_date) =2019 THEN 1 ELSE 0 END) as orders_in_2019
FROM Orders as o
RIGHT JOIN Users as u
ON o.buyer_id = u.user_id
GROUP BY u.user_id
```

### 1159. Market Analysis II

```mysql
-- SQL server
SELECT u.user_id AS seller_id,
    CASE WHEN u.favorite_brand=a.item_brand THEN 'yes' ELSE 'no' END AS '2nd_item_fav_brand'
FROM users u
LEFT JOIN 
    (SELECT buyer_id, seller_id, i.item_brand,
        RANK() OVER (PARTITION BY seller_id ORDER BY order_date) AS rnk
    FROM orders as o
    JOIN items i
    ON o.item_id = i.item_id) a
ON u.user_id=a.seller_id AND a.rnk=2
```

### 1164. Product Price at a Given Date

```mysql
SELECT id.product_id, IFNULL(b.new_price, 10) AS price
FROM (
    SELECT DISTINCT product_id
    FROM products) id
LEFT JOIN (
    SELECT p.product_id, p.new_price, p.change_date
    FROM products p
    JOIN (
        SELECT product_id, MAX(change_date) as change_date
        FROM products
        WHERE change_date<='2019-08-16'
        GROUP BY product_id) a
    ON p.product_id=a.product_id AND p.change_date=a.change_date) b
ON id.product_id=b.product_id
```